In [152]:
from contextlib import contextmanager
from functools import wraps, update_wrapper, partial
import time

In [127]:
class Timed:
    
    def __init__(self, func=None):
        print('in init')
        if func:
            self.func = func
            update_wrapper(self, func)
        
    def __call__(self, *args, **kwargs):
        print('in call')
        self.__enter__()
        res = self.func(*args, **kwargs)
        self.__exit__(None, None, None)
        return res
    
    def __enter__(self):
        self.start = time.time()
        print('enter')
    
    def __exit__(self, exc_type, exc_value, traceback):
        print('exit:', time.time() - self.start)
        self.start = None

## Approach 1

Issues
- non-strict mode doesn't work
- bit fuzzy re how `__call__` is working. Weird that calling exit() with None works.

In [218]:
class ContextDecorator:
    
    def __init__(self, func=None):
        print('parent init')
        if func:
            self.func = func
            update_wrapper(self, func)
    
    def __call__(self, *args, **kwargs):
        print('parent call')
        self.__enter__()
        res = self.func(*args, **kwargs)
        self.__exit__(None, None, None)
        return res

In [219]:
class NewTimed(ContextDecorator):
    
    def __init__(self, func=None):
        super().__init__(func)
        
    def __enter__(self):
        self.start = time.time()
        print('enter')
    
    def __exit__(self, exc_type, exc_value, traceback):
        print('exit:', time.time() - self.start)
        self.start = None

In [220]:
@Timed
def foo(a, b=1, c='c'):
    """foo docs"""
    for i in range(1, b+1):
        time.sleep(1)
        a += i
    return a

in init


In [221]:
print(foo(3))

in call
enter
exit: 1.002798080444336
4


In [132]:
with Timed():
    print(foo(5))

in init
enter
in call
enter
exit: 1.0021421909332275
6
exit: 1.0025231838226318


In [142]:
with Timed():
    a = 1
    for i in range(10_000_000):
        a += 1
    print(a)

in init
enter
10000001
exit: 1.1956799030303955


In [135]:
with Timed():
    a = 1
    for i in range(10_000_000):
        a += 1
    print(a)
    print(a + 'a')

in init
enter
10000001
exit: 1.0558419227600098


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [222]:
@NewTimed
def new_foo(a, b=1, c='c'):
    """foo docs"""
    for i in range(1, b+1):
        time.sleep(1)
        a += i
    return a

parent init


In [223]:
new_foo(31)

parent call
enter
exit: 1.0000977516174316


32

In [224]:
with NewTimed():
    a = 1
    for i in range(10_000_000):
        a += 1
    print(a)

parent init
enter
10000001
exit: 0.8550708293914795


In [225]:
import signal
import warnings

### reproduce timebox

In [248]:
class TimeExceededError(Exception):
    pass

def timebox_handler(time, frame):
    raise TimeExceededError('Time limit exceeded.')

@contextmanager
def timebox(time, strict=True):
    try:
        signal.signal(signal.SIGALRM, timebox_handler)
        signal.alarm(time)
        yield
    except Exception as e:
        if strict: raise
        warnings.warn(e.args[0])
    finally:
        signal.alarm(0)

def timeboxed(time, strict=True):
    def intermediate_wrapper(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            with timebox(time, strict) as tb:
                return func(*args, **kwargs)
        return wrapper
    return intermediate_wrapper

In [246]:
class Timebox(ContextDecorator):
    
    def __init__(self, time, strict=True):
        print('child init')
        self.time = time
        self.strict = strict
        
    def __call__(self, func, *args, **kwargs):
        if not hasattr(self, 'func'):
            print('child call: in if')
            super().__init__(func)
            return self.__call__
        
        print('child call: after if')
        args = [func] + list(args)
        return super().__call__(*args, **kwargs)
    
    def __enter__(self):
        print('enter')
        signal.signal(signal.SIGALRM, timebox_handler)
        signal.alarm(self.time)
        
    def __exit__(self, exc_type, exc_value, traceback):
        print('exit')
        if exc_type:
            if self.strict: raise
            warnings.warn(exc_type)
        signal.alarm(0)

In [240]:
@Timebox(1)
def bar(a, b=True, c=3, **kwargs):
    print('start bar')
    time.sleep(a)
    print('end bar')
    return a*c

child init
child call: in if
parent init


In [241]:
bar(0.5)

child call: after if
parent call
enter
start bar
end bar
exit


1.5

In [242]:
bar(2)

child call: after if
parent call
enter
start bar


TimeExceededError: Time limit exceeded.

In [243]:
with Timebox(2):
    print('start inside')
    time.sleep(1)
    print('end inside')

child init
enter
start inside
end inside
exit


In [244]:
with Timebox(2):
    print('start inside')
    time.sleep(3)
    print('end inside')

child init
enter
start inside
exit


TimeExceededError: Time limit exceeded.

In [247]:
with Timebox(2, False):
    print('start inside')
    time.sleep(3)
    print('end inside')

child init
enter
start inside
exit


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: <class '__main__.TimeExceededError'>


TimeExceededError: Time limit exceeded.

## Approach 2

Deals with decorators that accept arguments.

Issues

- Non-strict mode still doesn't work
- Still don't quite understand how call() is calling exit()

In [258]:
class ContextDecoratorNew:
    
    def __init__(self, func=None):
        print('parent init')
        if func: self._wrap_func(func)
    
    def __call__(self, *args, **kwargs):
        print('parent call')
        if not hasattr(self, 'func'):
            self._wrap_func(args[0])
            return self.__call__

        print('parent call (post if)')
        self.__enter__()
        res = self.func(*args, **kwargs)
        self.__exit__(None, None, None)
        return res
    
    def _wrap_func(self, func):
        self.func = func
        update_wrapper(self, func)

In [269]:
class TimeboxNew(ContextDecoratorNew):
    
    def __init__(self, time, strict=True):
        print('child init')
        self.time = time
        self.strict = strict
        
    def __call__(self, *args, **kwargs):
        print('child call')
        return super().__call__(*args, **kwargs)
    
    def __enter__(self):
        print('child enter')
        signal.signal(signal.SIGALRM, timebox_handler)
        signal.alarm(self.time)
        
    def __exit__(self, exc_type, exc_value, traceback):
        print('child exit', type(exc_type), type(exc_value), type(traceback))
        if exc_type:
            if self.strict: raise
            warnings.warn(exc_type)
        signal.alarm(0)

In [280]:
@TimeboxNew(1)
def bar_new(a, b=True, c=3, **kwargs):
    print('start bar')
    time.sleep(a)
    print('end bar')
    return a*c

child init
child call
parent call


In [281]:
bar_new(2)

child call
parent call
parent call (post if)
child enter
start bar


TimeExceededError: Time limit exceeded.

In [282]:
bar_new(.5)

child call
parent call
parent call (post if)
child enter
start bar
end bar
child exit <class 'NoneType'> <class 'NoneType'> <class 'NoneType'>


1.5

In [283]:
with TimeboxNew(2):
    print('start inside')
    time.sleep(1)
    print('end inside')

child init
child enter
start inside
end inside
child exit <class 'NoneType'> <class 'NoneType'> <class 'NoneType'>


In [284]:
with TimeboxNew(1):
    print('start inside')
    time.sleep(2)
    print('end inside')

child init
child enter
start inside
child exit <class 'type'> <class '__main__.TimeExceededError'> <class 'traceback'>


TimeExceededError: Time limit exceeded.

In [285]:
with TimeboxNew(1, False):
    print('start inside')
    time.sleep(2)
    print('end inside')

child init
child enter
start inside
child exit <class 'type'> <class '__main__.TimeExceededError'> <class 'traceback'>


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: <class '__main__.TimeExceededError'>


TimeExceededError: Time limit exceeded.